# 20NG (Twenty Newsgroups). Preprocessing

Here goes an example of data preprocessing and converting it to TopicNet's Dataset format.

* Example of a toy dataset: [test_dataset.csv](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/tests/test_data/test_dataset.csv)
* Dataset source file (with some explanations in docstring): [dataset.py](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/cooking_machine/dataset.py)

# Contents<a id="contents"></a>

* [Loading data](#data-loading)
* [Preparing data](#data-preparation)

In [45]:
import numpy as np
import pandas as pd
import re
import shutil
import string

from collections import Counter
from glob import glob

from sklearn import datasets
from sklearn.datasets import fetch_20newsgroups

In [ ]:
import nltk

from nltk.collocations import (
    BigramAssocMeasures,
    BigramCollocationFinder,
)
from nltk.corpus import (
    stopwords,
    wordnet,
)
from nltk.stem import WordNetLemmatizer

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import cm

## Loading data<a id="data-loading"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Let's download the dataset:

In [4]:
train_20 = fetch_20newsgroups(
    subset='train',
    remove=('headers', 'footers', 'quotes'),
)
test_20 = fetch_20newsgroups(
    subset='test',
    remove=('headers', 'footers', 'quotes'),
)

In [5]:
train_20.pop('DESCR')
labels = train_20.pop('target_names')

for k in train_20.keys():
    print(len(train_20[k]), k)

11314 data
11314 filenames
11314 target


In [6]:
test_20.pop('DESCR')
labels_test = test_20.pop('target_names')

for k in test_20.keys():
    print(len(test_20[k]), k)

7532 data
7532 filenames
7532 target


## Preparing data (lemmatization, Vowpal Wabbit & TopicNet's format)<a id="data-preparation"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Wrapping all in .csv files:

In [7]:
train_pd = pd.DataFrame(train_20).rename(columns = {'data':'raw_text'},)
# train_pd['raw_text'] = train_pd['raw_text'].apply(lambda x: x.decode('windows-1252'))
train_pd['id'] = train_pd.filenames.apply( lambda x: '.'.join(x.split('/')[-2:]).replace('.','_'))

test_pd = pd.DataFrame(test_20).rename(columns = {'data':'raw_text'})
# test_pd['raw_text'] = test_pd['raw_text'].apply(lambda x: x.decode('windows-1252'))
test_pd['id'] = test_pd.filenames.apply( lambda x: '.'.join(x.split('/')[-2:]))

Better to exclude these documents (one may look here [20-newsgroups-secrets](https://github.com/Alvant/20-newsgroups-secrets) for more details).

In [36]:
bad_names = [9976, 9977, 9978, 9979, 9980, 9981, 9982, 9983, 9984, 9985, 9986, 9987, 9988, 9990]
bad_names = [f"comp_os_ms-windows_misc_{i}" for i in bad_names]

bad_indices = train_pd.query("id in @bad_names").index

Below we define some functions for text preprocessing.

In [24]:
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return ''

In [ ]:
pattern = re.compile('\S*@\S*\s?')

In [26]:
def vowpalize_sequence(sequence):
    word_2_frequency = Counter(sequence)
    
    del word_2_frequency['']
    
    vw_string = ''
    
    for word in word_2_frequency:
        vw_string += word + ":" + str(word_2_frequency[word]) + ' '
    
    return vw_string

def do_vw_for_me_please(dataframe):
    bad_entries = []
    tokenized_text = []
    
    for indx, text in enumerate(dataframe['raw_text'].values):
        try:
            text = str(pattern.sub('', text))
        except TypeError:
            text=''
        
        tokens = [tok for tok in nltk.wordpunct_tokenize(text.lower()) if len(tok) > 1]
        tokenized_text.append(nltk.pos_tag(tokens))
    
    dataframe['tokenized'] = tokenized_text

    stop = set(stopwords.words('english'))

    lemmatized_text = []
    wnl = WordNetLemmatizer()
    
    for text in dataframe['tokenized'].values:
        lemmatized = [wnl.lemmatize(word, nltk2wn_tag(pos))
                      if nltk2wn_tag(pos) != ''
                      else wnl.lemmatize(word)
                      for word, pos in text ]
        lemmatized = [word for word in lemmatized 
                      if word not in stop and word.isalpha()]
        lemmatized_text.append(lemmatized)
    
    dataframe['lemmatized'] = lemmatized_text

    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_documents(dataframe['lemmatized'])
    finder.apply_freq_filter(5)
    set_dict = set(finder.nbest(bigram_measures.pmi,32100)[100:])
    documents = dataframe['lemmatized']
    bigrams = []

    for doc in documents:
        entry = ['_'.join([word_first, word_second])
                 for word_first, word_second in zip(doc[:-1],doc[1:])
                 if (word_first, word_second) in set_dict]
        bigrams.append(entry)

    dataframe['bigram'] = bigrams
    
    vw_text = []

    for index, data in dataframe.iterrows():
        vw_string = ''    
        doc_id = data.id
        lemmatized = '@lemmatized ' + vowpalize_sequence(data.lemmatized)
        bigram = '@bigram ' + vowpalize_sequence(data.bigram)
        vw_string = ' |'.join([doc_id, lemmatized, bigram])
        vw_text.append(vw_string)

    dataframe['vw_text'] = vw_text

    print('num bad entries ', len(bad_entries))
    print(bad_entries)

    return dataframe

And here are the final datasets!
Each row represents a document.
Columns `id`, `raw_text` and `vw_text` are required (look at this [toy dataset](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/tests/test_data/test_dataset.csv), for example).

In [41]:
train_pd = do_vw_for_me_please(train_pd)
display(train_pd.head())

test_pd = do_vw_for_me_please(test_pd)
display(test_pd.head())

num bad entries  0
[]


,raw_text,filenames,target,id,tokenized,lemmatized,bigram,vw_text
0,I was wondering if anyone out there could enli...,/home/bulatov/scikit_learn_data/20news_home/20...,7,rec_autos_102994,"[(was, VBD), (wondering, VBG), (if, IN), (anyo...","[wonder, anyone, could, enlighten, car, saw, d...","[wonder_anyone, anyone_could, sport_car, car_l...",rec_autos_102994 |@lemmatized wonder:1 anyone:...
1,A fair number of brave souls who upgraded thei...,/home/bulatov/scikit_learn_data/20news_home/20...,4,comp_sys_mac_hardware_51861,"[(fair, JJ), (number, NN), (of, IN), (brave, J...","[fair, number, brave, soul, upgrade, si, clock...","[clock_oscillator, please_send, top_speed, hea...",comp_sys_mac_hardware_51861 |@lemmatized fair:...
2,"well folks, my mac plus finally gave up the gh...",/home/bulatov/scikit_learn_data/20news_home/20...,4,comp_sys_mac_hardware_51879,"[(well, RB), (folks, NNS), (my, PRP$), (mac, J...","[well, folk, mac, plus, finally, give, ghost, ...","[mac_plus, life_way, way_back, market_new, new...",comp_sys_mac_hardware_51879 |@lemmatized well:...
3,\nDo you have Weitek's address/phone number? ...,/home/bulatov/scikit_learn_data/20news_home/20...,1,comp_graphics_38242,"[(do, VBP), (you, PRP), (have, VB), (weitek, V...","[weitek, address, phone, number, like, get, in...","[address_phone, phone_number, number_like, lik...",comp_graphics_38242 |@lemmatized weitek:1 addr...
4,"From article <C5owCB.n3p@world.std.com>, by to...",/home/bulatov/scikit_learn_data/20news_home/20...,14,sci_space_60880,"[(from, IN), (article, NN), (by, IN), (tom, NN...","[article, tom, baker, understanding, expected,...","[system_software, thing_check, introduce_new, ...",sci_space_60880 |@lemmatized article:1 tom:1 b...


num bad entries  0
[]


,raw_text,filenames,target,id,tokenized,lemmatized,bigram,vw_text
0,I am a little confused on all of the models of...,/home/bulatov/scikit_learn_data/20news_home/20...,7,rec.autos.103343,"[(am, VBP), (little, JJ), (confused, VBN), (on...","[little, confuse, model, bonnevilles, hear, le...","[could_someone, someone_tell, tell_difference,...",rec.autos.103343 |@lemmatized little:1 confuse...
1,I'm not familiar at all with the format of the...,/home/bulatov/scikit_learn_data/20news_home/20...,5,comp.windows.x.67445,"[(not, RB), (familiar, JJ), (at, IN), (all, DT...","[familiar, format, face, thingies, see, folk, ...","[get_see, make_one, one_get, seem_find, could_...",comp.windows.x.67445 |@lemmatized familiar:1 f...
2,"\nIn a word, yes.\n",/home/bulatov/scikit_learn_data/20news_home/20...,0,alt.atheism.53603,"[(in, IN), (word, NN), (yes, NN)]","[word, yes]",[],alt.atheism.53603 |@lemmatized word:1 yes:1 |...
3,\nThey were attacking the Iraqis to drive them...,/home/bulatov/scikit_learn_data/20news_home/20...,17,talk.politics.mideast.77355,"[(they, PRP), (were, VBD), (attacking, VBG), (...","[attack, iraqi, drive, kuwait, country, whose,...","[think_u, saudi_arabia, much_anything, saudi_a...",talk.politics.mideast.77355 |@lemmatized attac...
4,\nI've just spent two solid months arguing tha...,/home/bulatov/scikit_learn_data/20news_home/20...,19,talk.religion.misc.84194,"[(ve, NN), (just, RB), (spent, VBN), (two, CD)...","[spend, two, solid, month, argue, thing, objec...",[moral_system],talk.religion.misc.84194 |@lemmatized spend:1 ...


Saving to disk (TopicNet's [Dataset](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/cooking_machine/dataset.py) can be constructed using saved .csv file with text data).

In [44]:
! mkdir 20_News_dataset

train_pd.drop(bad_indices).to_csv('20_News_dataset/train_preprocessed.csv')
test_pd.to_csv('20_News_dataset/test_preprocessed.csv')